In [ ]:
using Plots
using LaTeXStrings

using Statistics

# Global constants 

In [ ]:
C_m = 9 * pi

E_Na = 115

E_K = -12

V_rest = 10.6

G_Na = 1080 * pi

G_K = 324 * pi

G_m = 2.7 * pi

g_A = 10
g_G = 10

k_p = 5
V_p = 62

E_A = 60
E_G = -20

alpha_A = 1.1
alpha_G = 5

beta_A = 0.19
beta_G = 0.3

# Empirical equations 

In [ ]:
function alpha_n(V)
    
    return (10 - V) / (100 * (exp((10 - V) / 10) - 1))
    
end

function beta_n(V)
   
    return 0.125 * exp(-V / 80)
    
end

function alpha_m(V)
    
    return (25 - V) / (10 * (exp((25 - V) / 10) - 1))
    
end

function beta_m(V)
   
    return 4 * exp(-V / 18)
    
end

function alpha_h(V)
   
    return 0.07 * exp(-V / 20)
    
end

function beta_h(V)
   
    return 1 / (exp((30 - V) / 10) + 1)
    
end

# Current input 

In [ ]:
function I(t, I0)
   
    return I0
    
end

# Diferential equations 

In [ ]:
function n_dot(n, V)
   
    return alpha_n(V) * (1 - n) - beta_n(V) * n
    
end

function m_dot(m, V)
   
    return alpha_m(V) * (1 - m) - beta_m(V) * m
    
end

function h_dot(h, V)
   
    return alpha_h(V) * (1 - h) - beta_h(V) * h
    
end

function I_A(r, V)
    
    return g_A * r * (E_A - V)
    
end

function I_G(r, V)
    
    return g_G * r * (E_G - V)
    
end

function T(V_pre)
   
    return 1 / (1 + exp(-(V_pre - V_p) / k_p))
    
end

function r_A_dot(r, V_pre)
   
    return alpha_A * T(V_pre) * (1 - r) - beta_A * r
    
end

function r_G_dot(r, V_pre)
   
    return alpha_G * T(V_pre) * (1 - r) - beta_G * r
    
end

function V_dot(V, n, m, h, I0, I_A, I_G, t)
   
    return (G_Na * m^3 * h * (E_Na - V) + G_K * n^4 * (E_K - V) + G_m * (V_rest - V) + I(t, I0) + I_A + I_G) / C_m
    
end

# Euler solver 

In [ ]:
@inbounds function unidirectional_excitatory(N, h_int, dt, I0, V0, n0, m0, h0)

    tf = Int(floor(N * dt))
    writting_period = Int(floor(dt / h_int))
    
    h_2 = 0.5 * h_int
    h_6 = h_int / 6
    
    data = zeros(11, N)
    
    data[1, 1] = 0 #t
    data[2, 1] = V0[1] #V1
    data[3, 1] = V0[2] #V2
    data[4, 1] = n0[2] #n2
    data[5, 1] = m0[2] #m2
    data[6, 1] = h0[2] #h2
    
    data[7, 1] = n0[1] #n1
    data[8, 1] = m0[1] #m1
    data[9, 1] = h0[1] #h1
    
    data[10, 1] = 0 #I excitatory
    data[11, 1] = 0 #I inhibitory
    
    V1 = V0[1]
    n1 = n0[1]
    m1 = m0[1]
    h1 = h0[1]
    
    V2 = V0[2]
    n2 = n0[2]
    m2 = m0[2]
    h2 = h0[2]
    
    r_A = 0
    r_G = 0
    
    I_A = 0
    I_G = 0
    
    i = 1
    
    t = 0
    
    while i < N
        
        for k in 1 : writting_period
            
            V1_pre = V1
            V2_pre = V2
            
            #Neuron 1
            n1 += h_int * n_dot(n1, V1)
            m1 += h_int * m_dot(m1, V1)
            h1 += h_int * h_dot(h1, V1)
            
            V1 += h_int * V_dot(V1, n1, m1, h1, I0, 0, 0, t)
            
            #Neuron 2
            n2 += h_int * n_dot(n2, V2)
            m2 += h_int * m_dot(m2, V2)
            h2 += h_int * h_dot(h2, V2)
            
            r_A += h_int * r_A_dot(r_A, V1_pre) 
            r_G += h_int * r_G_dot(r_G, V1_pre)
            
            I_A = g_A * r_A * (E_A - V2)
            I_G = 0 #g_G * r_G * (E_G - V2)
            
            V2 += h_int * V_dot(V2, n2, m2, h2, I0, I_A, I_G, t)
            
            t += h_int
            
        end
            
        i += 1
        
        data[1, i]= t
        data[2, i] = V1
        data[3, i] = V2
        
        data[4, i] = n2
        data[5, i] = m2
        data[6, i] = h2
        
        data[7, i] = n1
        data[8, i] = m1
        data[9, i] = h1
        
        data[10, i] = I_A 
        data[11, i] = I_G 
        
    end
        
    return data
    
end

@inbounds function unidirectional_inhibitory(N, h_int, dt, I0, V0, n0, m0, h0)

    tf = Int(floor(N * dt))
    writting_period = Int(floor(dt / h_int))
    
    h_2 = 0.5 * h_int
    h_6 = h_int / 6
    
    data = zeros(11, N)
    
    data[1, 1] = 0 #t
    data[2, 1] = V0[1] #V1
    data[3, 1] = V0[2] #V2
    data[4, 1] = n0[2] #n2
    data[5, 1] = m0[2] #m2
    data[6, 1] = h0[2] #h2
    
    data[7, 1] = n0[1] #n1
    data[8, 1] = m0[1] #m1
    data[9, 1] = h0[1] #h1
    
    data[10, 1] = 0 #I excitatory
    data[11, 1] = 0 #I inhibitory
    
    V1 = V0[1]
    n1 = n0[1]
    m1 = m0[1]
    h1 = h0[1]
    
    V2 = V0[2]
    n2 = n0[2]
    m2 = m0[2]
    h2 = h0[2]
    
    r_A = 0
    r_G = 0
    
    I_A = 0
    I_G = 0
    
    i = 1
    
    t = 0
    
    while i < N
        
        for k in 1 : writting_period
            
            V1_pre = V1
            V2_pre = V2
            
            #Neuron 1
            n1 += h_int * n_dot(n1, V1)
            m1 += h_int * m_dot(m1, V1)
            h1 += h_int * h_dot(h1, V1)
            
            V1 += h_int * V_dot(V1, n1, m1, h1, I0, 0, 0, t)
            
            #Neuron 2
            n2 += h_int * n_dot(n2, V2)
            m2 += h_int * m_dot(m2, V2)
            h2 += h_int * h_dot(h2, V2)
            
            r_A += h_int * r_A_dot(r_A, V1_pre) 
            r_G += h_int * r_G_dot(r_G, V1_pre)
            
            I_A = 0#g_A * r_A * (E_A - V2)
            I_G = g_G * r_G * (E_G - V2)
            
            V2 += h_int * V_dot(V2, n2, m2, h2, I0, I_A, I_G, t)
            
            t += h_int
            
        end
            
        i += 1
        
        data[1, i]= t
        data[2, i] = V1
        data[3, i] = V2
        
        data[4, i] = n2
        data[5, i] = m2
        data[6, i] = h2
        
        data[7, i] = n1
        data[8, i] = m1
        data[9, i] = h1
        
        data[10, i] = I_A 
        data[11, i] = I_G 
        
    end
        
    return data
    
end

@inbounds function mutually_excitatory(N, h_int, dt, I0, V0, n0, m0, h0)

    tf = Int(floor(N * dt))
    writting_period = Int(floor(dt / h_int))
    
    h_2 = 0.5 * h_int
    h_6 = h_int / 6
    
    data = zeros(11, N)
    
    data[1, 1] = 0 #t
    data[2, 1] = V0[1] #V1
    data[3, 1] = V0[2] #V2
    data[4, 1] = n0[2] #n2
    data[5, 1] = m0[2] #m2
    data[6, 1] = h0[2] #h2
    
    data[7, 1] = n0[1] #n1
    data[8, 1] = m0[1] #m1
    data[9, 1] = h0[1] #h1
    
    data[10, 1] = 0 #I excitatory
    data[11, 1] = 0 #I inhibitory
    
    V1 = V0[1]
    n1 = n0[1]
    m1 = m0[1]
    h1 = h0[1]
    
    V2 = V0[2]
    n2 = n0[2]
    m2 = m0[2]
    h2 = h0[2]
    
    r_A1 = 0
    r_A2 = 0
    
    I_A = 0
    I_G = 0
    
    i = 1
    
    t = 0
    
    while i < N
        
        for k in 1 : writting_period
            
            V1_pre = V1
            V2_pre = V2
            
            #Neuron 1
            n1 += h_int * n_dot(n1, V1)
            m1 += h_int * m_dot(m1, V1)
            h1 += h_int * h_dot(h1, V1)
            
            r_A1 += h_int * r_A_dot(r_A1, V2_pre) 
            
            I_A1 = g_A * r_A1 * (E_A - V1)
 
            
            V1 += h_int * V_dot(V1, n1, m1, h1, I0, I_A1, 0, t)
            
            #Neuron 2
            n2 += h_int * n_dot(n2, V2)
            m2 += h_int * m_dot(m2, V2)
            h2 += h_int * h_dot(h2, V2)
            
            r_A2 += h_int * r_A_dot(r_A2, V1_pre) 
            
            I_A2 = g_A * r_A2 * (E_A - V2)
            
            V2 += h_int * V_dot(V2, n2, m2, h2, I0, I_A2, 0, t)
            
            t += h_int
            
        end
            
        i += 1
        
        data[1, i]= t
        data[2, i] = V1
        data[3, i] = V2
        
        data[4, i] = n2
        data[5, i] = m2
        data[6, i] = h2
        
        data[7, i] = n1
        data[8, i] = m1
        data[9, i] = h1
        
        data[10, i] = I_A 
        data[11, i] = I_G 
        
    end
        
    return data
    
end

@inbounds function mutually_inhibitory(N, h_int, dt, I0, V0, n0, m0, h0)

    tf = Int(floor(N * dt))
    writting_period = Int(floor(dt / h_int))
    
    h_2 = 0.5 * h_int
    h_6 = h_int / 6
    
    data = zeros(11, N)
    
    data[1, 1] = 0 #t
    data[2, 1] = V0[1] #V1
    data[3, 1] = V0[2] #V2
    data[4, 1] = n0[2] #n2
    data[5, 1] = m0[2] #m2
    data[6, 1] = h0[2] #h2
    
    data[7, 1] = n0[1] #n1
    data[8, 1] = m0[1] #m1
    data[9, 1] = h0[1] #h1
    
    data[10, 1] = 0 #I excitatory
    data[11, 1] = 0 #I inhibitory
    
    V1 = V0[1]
    n1 = n0[1]
    m1 = m0[1]
    h1 = h0[1]
    
    V2 = V0[2]
    n2 = n0[2]
    m2 = m0[2]
    h2 = h0[2]
    
    r_G1 = 0
    r_G2 = 0
    
    I_A = 0
    I_G = 0
    
    i = 1
    
    t = 0
    
    while i < N
        
        for k in 1 : writting_period
            
            V1_pre = V1
            V2_pre = V2
            
            #Neuron 1
            n1 += h_int * n_dot(n1, V1)
            m1 += h_int * m_dot(m1, V1)
            h1 += h_int * h_dot(h1, V1)
            
            r_G1 += h_int * r_G_dot(r_G1, V2_pre) 
            
            I_G1 = g_G * r_G1 * (E_G - V1)
 
            
            V1 += h_int * V_dot(V1, n1, m1, h1, I0, 0, I_G1, t)
            
            #Neuron 2
            n2 += h_int * n_dot(n2, V2)
            m2 += h_int * m_dot(m2, V2)
            h2 += h_int * h_dot(h2, V2)
            
            r_G2 += h_int * r_G_dot(r_G2, V1_pre) 
            
            I_G2 = g_G * r_G2 * (E_G - V2)
            
            V2 += h_int * V_dot(V2, n2, m2, h2, I0, 0, I_G2, t)
            
            t += h_int
            
        end
            
        i += 1
        
        data[1, i]= t
        data[2, i] = V1
        data[3, i] = V2
        
        data[4, i] = n2
        data[5, i] = m2
        data[6, i] = h2
        
        data[7, i] = n1
        data[8, i] = m1
        data[9, i] = h1
        
        data[10, i] = I_A 
        data[11, i] = I_G 
        
    end
        
    return data
    
end

@inbounds function mutually_excitatory_inhibitory(N, h_int, dt, I0, V0, n0, m0, h0)

    tf = Int(floor(N * dt))
    writting_period = Int(floor(dt / h_int))
    
    h_2 = 0.5 * h_int
    h_6 = h_int / 6
    
    data = zeros(11, N)
    
    data[1, 1] = 0 #t
    data[2, 1] = V0[1] #V1
    data[3, 1] = V0[2] #V2
    data[4, 1] = n0[2] #n2
    data[5, 1] = m0[2] #m2
    data[6, 1] = h0[2] #h2
    
    data[7, 1] = n0[1] #n1
    data[8, 1] = m0[1] #m1
    data[9, 1] = h0[1] #h1
    
    data[10, 1] = 0 #I excitatory
    data[11, 1] = 0 #I inhibitory
    
    V1 = V0[1]
    n1 = n0[1]
    m1 = m0[1]
    h1 = h0[1]
    
    V2 = V0[2]
    n2 = n0[2]
    m2 = m0[2]
    h2 = h0[2]
    
    r_A1 = 0
    r_G2 = 0
    
    I_A = 0
    I_G = 0
    
    i = 1
    
    t = 0
    
    while i < N
        
        for k in 1 : writting_period
            
            V1_pre = V1
            V2_pre = V2
            
            #Neuron 1
            n1 += h_int * n_dot(n1, V1)
            m1 += h_int * m_dot(m1, V1)
            h1 += h_int * h_dot(h1, V1)
            
            r_A1 += h_int * r_A_dot(r_A1, V2_pre) 
            
            I_A1 = g_A * r_A1 * (E_A - V1)
 
            
            V1 += h_int * V_dot(V1, n1, m1, h1, I0, I_A1, 0, t)
            
            #Neuron 2
            n2 += h_int * n_dot(n2, V2)
            m2 += h_int * m_dot(m2, V2)
            h2 += h_int * h_dot(h2, V2)
            
            r_G2 += h_int * r_G_dot(r_G2, V1_pre) 
            
            I_G2 = g_G * r_G2 * (E_G - V2)
            
            V2 += h_int * V_dot(V2, n2, m2, h2, I0, 0, I_G2, t)
            
            t += h_int
            
        end
            
        i += 1
        
        data[1, i]= t
        data[2, i] = V1
        data[3, i] = V2
        
        data[4, i] = n2
        data[5, i] = m2
        data[6, i] = h2
        
        data[7, i] = n1
        data[8, i] = m1
        data[9, i] = h1
        
        data[10, i] = I_A 
        data[11, i] = I_G 
        
    end
        
    return data
    
end

# Run simulation 

In [ ]:
N = 2 * 10^5

h_int = 0.001
dt = h_int

I0 = 280

V0 = [0, 0]
n0 = [0.5, 0.5]
m0 = [0.5, 0.5]
h0 = [0.5, 0.5]

#=
n0 = rand(2)
m0 = rand(2)
h0 = rand(2)
=#

#result = unidirectional_excitatory(N, h_int, dt, I0, V0, n0, m0, h0)
#result = unidirectional_inhibitory(N, h_int, dt, I0, V0, n0, m0, h0)
result = mutually_excitatory(N, h_int, dt, I0, V0, n0, m0, h0)
#result = mutually_inhibitory(N, h_int, dt, I0, V0, n0, m0, h0)
#result = mutually_excitatory_inhibitory(N, h_int, dt, I0, V0, n0, m0, h0)

# Potential in time $V(t)$

In [ ]:
plot(result[1, :], result[2, :], label=L"Excitatory", legend=:topright, size=(800, 600), 
    xlabel=L"$t \ [ms]$", ylabel=L"$V \ [mV]$", lw=1)

plot!(result[1, :], result[3, :], label=L"Excitatory", legend=:topleft, size=(800, 600), 
    xlabel=L"$t \ [ms]$", ylabel=L"$V \ [mV]$", guidefont=16, legendfont=12)

#savefig("mutually_excitatory_rand.png")

# Ionic currents

In [ ]:
I_Na = @. G_Na * result[5, :]^3 * result[6, :] * (result[3, :] - E_Na) 
I_K = @. G_K * result[4, :]^4 * (result[3, :] - E_K);
I_L = @. G_m * (V_rest - result[3, :]);

I_Na_1 = @. G_Na * result[8, :]^3 * result[9, :] * (result[2, :] - E_Na) 
I_K_1 = @. G_K * result[7, :]^4 * (result[2, :] - E_K);
I_L_1 = @. G_m * (V_rest - result[2, :]);

In [ ]:
plot(result[1, :], I_Na, color="blue", label=L"I_{Na}", legend=:topleft, size=(800, 600))

plot!(result[1, :], I_K, color="red", label=L"I_K")
plot!(result[1, :], I_L, color="green", label=L"I_L")

plot!(result[1, :], I_Na_1, color="blue", label="", linestyle=:dash)
plot!(result[1, :], I_K_1, color="red",  label="", linestyle=:dash)
plot!(result[1, :], I_L_1, color="green", label="", linestyle=:dash)

plot!(result[1, :], result[10, :].*20, color="black", label=L"I_A (\mathrm{x} \ 20)")
#plot!(result[1, :], result[11, :].*20, color="black", label=L"I_G (\mathrm{x} \ 20)")

plot!(xlim=(112, 120), xlabel=L"$t \ [ms]$", ylabel=L"$I \ [\mu A]$", lw=1, guidefont=16, legendfont=12, grid=true)

#savefig("currents_mutual_excitatory.png")